# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
2. Write vectorized code whenever possible.
3. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
4. Write your functions in the provided `hw2.py` python module only. All the logic you write is imported and used in this jupyter notebook.
5. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. Any other imports detected in `hw2.py` will earn you the grade of 0, even if you only used them for testing.
6. Your code must run without errors. During the environment setup, you were given a specific version of `numpy` to install. Changes of the configuration we provided are at your own risk. Code that cannot run will also earn you the grade of 0.
7. Write your own code. Cheating will not be tolerated. 
8. Submission includes the `hw2.py` file and this notebook. Answers to qualitative questions should be written in markdown cells (with $\LaTeX$ support).
9. You are allowed to include additional functions.
10. Submission: zip only the completed jupyter notebook and the python file `hw2.py`. Do not include the data or any directories. Name the file `ID1_ID2.zip` and submit only one copy of the assignment.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Implement a decision tree from scratch.
4. Prune the tree to achieve better results.
5. Visualize your results and the tree.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decision_trees import * # this imports all functions from hw2.

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Warmup - OOP in python

Our decision tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [7]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)
        

In [8]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children


[<__main__.Node at 0x10f17a080>, <__main__.Node at 0x10f17a0f0>]

## Data preprocessing

We will use the breast cancer dataset that is available as a part of sklearn - a popular machine learning and data science library in python. In this example, our dataset will be a single matrix with the **labels on the last column**. Notice that you are not allowed to use additional functions from sklearn.

In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# load data set
X, y = datasets.load_breast_cancer(return_X_y = True)
X = np.column_stack([X,y]) # the last column holds the labels

# split data set
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)


Training dataset shape:  (426, 31)
Testing dataset shape:  (143, 31)


## Impurity Measures

Implement the functions `calc_gini` (5 points) and `calc_entropy` (5 points) in the python file `hw2.py`. You are encouraged to test your implementation using the cell below.

In [11]:
##### Your code here #####

## Building a Decision Tree

Use a Python class to construct the decision tree (look at the `DecisionNode` class in the python file `hw2.py`. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes continuous data. In this exercise, create at most a single split for each node of the tree. The threshold you need to use for this exercise are the average of each consecutive pair of values. For example, assume some features contains the following values: [1,2,3,4,5]. You should use the following thresholds [1.5, 2.5, 3.5, 4.5]. 
4. When constructing the tree, test all possible thresholds for each feature. The stopping criteria is a pure tree.

Complete the class `DecisionNode` in the python file `hw2.py`. The structure of this class is entirely up to you. Complete the function `build_tree` in the python file `hw2.py`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points).

In [15]:
# python support passing a function as arguments to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini, chi_value=1) 
tree_entropy = build_tree(data=X_train, impurity=calc_entropy, chi_value=1)


## Tree evaluation

Complete the functions `predict` and `calc_accuracy` in the python file `hw2.py`. You are allowed to implement this functionality as a class method.

After building both trees using the training set (using Gini and Entropy as impurity measures), you should calculate the accuracy on the test set and print the measure that gave you the best test accuracy. For the rest of the exercise, use that impurity measure. (10 points)

In [19]:
gini_accuracy = calc_accuracy(tree_gini, X_test)
entropy_accuracy = calc_accuracy(tree_entropy, X_test)
if gini_accuracy > entropy_accuracy:
    best_impurity = calc_gini
    print("Gini:")
    print(gini_accuracy)
else:
    best_impurity = calc_entropy
    print("Entropy:")
    print(entropy_accuracy)
    

Entropy:
93.7062937062937


## Chi square pre-pruning

Consider the following p-value cut-off values: [1 (no pruning), 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00001]. For each value, construct a tree and prune it according to the cut-off value. Next, calculate the training and testing accuracy. On a single plot, draw the training and testing accuracy as a function of the p-value. What p-value gives you the best results? Does the results support the theory you learned in class regarding Chi square pruning? Explain. (20 points)

**Note**: You need to change the `DecisionNode` to support Chi square pruning. Make sure the `chi_value=1` corresponds to no pruning. The values you need from the Chi square table are available in the python file `hw2.py`.

In [24]:
training = []
testing  = []

chi_values = [1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00001]
for chi_value in chi_values:
    tree = build_tree(X_train, best_impurity, chi_value)
    training.append(calc_accuracy(tree, X_train))
    testing.append(calc_accuracy(tree, X_test))
    

In [ ]:
plt.plot(chi_values, training, label= 'training')
plt.plot(chi_values, testing, label = 'testing')
plt.title('Accuracy as a function of P-Values')
plt.ylabel('Accuracy')
plt.xlabel('Chi-Value')
plt.xscale('log')
plt.legend()
plt.show()


Your answer here

## Post pruning

Construct a decision tree without Chi square pruning. For each leaf in the tree, calculate the test accuracy of the tree assuming no split occurred on the parent of that leaf and find the best such parent (in the sense that not splitting on that parent results in the best testing accuracy among possible parents). Make that parent into a leaf and repeat this process until you are left with just the root. On a single plot, draw the training and testing accuracy as a function of the number of internal nodes in the tree. Explain the results: what would happen to the training and testing accuracies when you remove nodes from the tree? Can you suggest a different approach to achieve better results? (20 points)

In [ ]:
tree = build_tree(X_train, best_impurity, 1)

num_of_nodes = []
training = []
testing = []

while not isinstance(tree, tuple):
    current_num_of_nodes = count_nodes(tree)
    queue = [tree]
    best_testing = 0
    best_training = 0
    best_num_of_nodes = 0
    best_node = 0
    best_parent = 0
    parents = [-1]
    while queue:
        if not isinstance(queue[0].children[0], tuple) and not isinstance(queue[0].children[1], tuple):
            queue.append(queue[0].children[0])
            parents.append((queue[0], 0))
            queue.append(queue[0].children[1])
            parents.append((queue[0], 1))
        else:
            if not isinstance(queue[0].children[0], tuple):
                queue.append(queue[0].children[0])
                parents.append((queue[0], 0))
            elif not isinstance(queue[0].children[1], tuple):
                queue.append(queue[0].children[1])
                parents.append((queue[0], 1))
            branch = queue[0]
            if parents[0] == -1:
                tree = (queue[0].majority, queue[0].num_of_instances)
            else:
                (parents[0])[0].children[(parents[0])[1]] = (queue[0].majority,
                                                         (parents[0])[0].
                                                         children[(parents[0])[1]].num_of_instances)
            testing_accuracy = calc_accuracy(tree, X_test)
            if testing_accuracy > best_testing:
                best_testing = testing_accuracy
                best_training = calc_accuracy(tree, X_train)
                best_num_of_nodes = current_num_of_nodes - count_nodes(branch) + 1
                best_node = queue[0]
                best_parent = parents[0]
            if parents[0] == -1:
                tree = branch
            else:
                (parents[0])[0].children[(parents[0])[1]] = branch
        del parents[0]
        del queue[0]
    training.append(best_training)
    testing.append(best_testing)
    num_of_nodes.append(best_num_of_nodes)
    if best_parent == -1:
        tree = (best_node.majority, best_node.num_of_instances)
    else:
        best_parent[0].children[best_parent[1]] = (best_node.majority, best_node.num_of_instances)

plt.plot(num_of_nodes, training, label= 'training')
plt.plot(num_of_nodes, testing, label = 'testing')
plt.title('Accuracy as a function of the number of internal nodes')
plt.ylabel('Accuracy')
plt.xlabel('Number Of Internal Nodes')
plt.legend()
plt.show()


NameError: name 'X_train' is not defined

Your answer here

## Print the tree

Complete the function `print_tree` in the python file `hw2.py` and print the tree using the chosen impurity measure and no pruning. Your code should like something like this (10 points):
```
[X0 <= 1],
  [X1 <= 2]
    [X2 <= 3], 
       leaf: [{1.0: 10}]
       leaf: [{0.0: 10}]
    [X4 <= 5], 
       leaf: [{1.0: 5}]
       leaf: [{0.0: 10}]
   leaf: [{1.0: 50}]
```


In [ ]:
print_tree(build_tree(X_train, best_impurity, 1), 1)


